In [ ]:
!pip install researchpy

In [ ]:
import seaborn as sns
import researchpy as rp
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
import warnings

# Simple T-Test

##### Simulating P-values

Suponhamos que você tenha um website com um banner de propaganda, e que 20% das pessoas que visitam o site clicam neste banner. A area de comunicação desenvolveu um novo banner e, ao implantar este novo banner, dos primeiros 30 clientes que o viram, 
11 pessoas clicaram no banner novo.

Como podemos descobrir se isso é um efeito aleatório ou um impacto real?

alpha --> 0.05  (significance level = 1-alpha = 95%)

In [ ]:
p = 11/30
primeiros_30 = np.random.choice([0, 1], p=[1-p, p], size=30)

In [ ]:
primeiros_30[0:5]

In [ ]:
sum(primeiros_30)

In [ ]:
primeiros_30.mean()

In [ ]:
dist_pop = np.array([np.mean(np.random.choice([0, 1], p=[0.8, 0.2], size=30)) for i in range(0, 1000)])

In [ ]:
sns.histplot(dist_pop)

In [ ]:
len(dist_pop[dist_pop >= p])/1000

Se a taxa real era de 20%, a chance de observarmos 11 cliques em 30 acessos (uma taxa de 36,67%) seria de 2,8%

In [ ]:
from scipy import stats

In [ ]:
cliques_novo_banner = [1] * 11 + [0] * (30-11)

In [ ]:
cliques_novo_banner[0:16]

In [ ]:
len(cliques_novo_banner)

In [ ]:
np.mean(cliques_novo_banner)

In [ ]:
stats.ttest_1samp(cliques_novo_banner, 0.02)

Como podemos ver acima, com essa amostra (11 positivos em 30 testes) **rejeitamos a hipótese nula**, ou seja, **a amostra observada tem média diferente da média da população**

# Paired T-Test

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from scipy import stats

https://raw.githubusercontent.com/Opensourcefordatascience/Data-sets/master/blood_pressure.csv

This dataset contains blood pressure readings before and after an <b>intervention</b>. These are variables “bp_before” and “bp_after”.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Opensourcefordatascience/Data-sets/master/blood_pressure.csv')

In [ ]:
df.head()

## The hypothesis being tested

* __Null hypothesis (H0): u1 = u2, which translates to the mean of sample 01 is equal to the mean of sample 02__
* __Alternative hypothesis (H1): u1 != u2, which translates to the means of sample 01 is not equal to sample 02__ 

## Assumption check 

- [ ]  The samples are independently and randomly drawn

**AND**

- [ ]  The distribution of the residuals between the two groups should follow the normal distribution

**AND**

- [ ]  The variances between the two groups are equal

**OR**

- [ ]  Approximately equal sample sizes

https://www.youtube.com/watch?v=OyB_w4XNQ58&t=315s

### Checking Normal distribution by Q-Q plot graph
https://www.statisticshowto.datasciencecentral.com/assumption-of-normality-test/

In [ ]:
df['diff_'] = df.bp_before-df.bp_after

In [ ]:
sns.displot(df['diff_'])

In [ ]:
plt.figure(figsize=(10,10))
stats.probplot(df['diff_'], plot=plt)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
stats.probplot(df['diff_']**2, plot=plt)
plt.show()

**Note:-** The corresponding points lies very close to line that means are our sample data sets are normally distributed

### Checking Normal distribution by method of `Shapiro stats`
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html

In [ ]:
stats.shapiro(df['diff_'])

### Checking if the variance is equal between samples

In [ ]:
df['bp_after'].var()

In [ ]:
df['bp_after'].var()

In [ ]:
stats.levene(df.bp_after, df.bp_before)

O teste-t é robusto à violações das premissas - mais robustos que os testes de normalidade e igualdade de variância. Portanto, embora devamos tomar cuidado com a utilização em casos extremos (especialmente com amostras pequenas), desvios das premissas do teste são toleráveis.

---

# Applications of Hypothesis Testing

Testando a efetividade do tratamento.

In [ ]:
df[['bp_before', 'bp_after']].describe()

In [ ]:
df[['bp_before', 'bp_after']].plot(kind='hist', alpha=0.7)

In [ ]:
stats.ttest_rel(df.bp_before, df.bp_after)

In [ ]:
stats.ttest_1samp(df.bp_before-df.bp_after, 0)

Como o valor p está abaixo de 0,05 podemos rejeitar a hipótese nula, ou seja, que pressão arterial antes da intervenção tem a mesma média que a pressão arterial depois da intervenção!

# Two Sample T-Test

In [ ]:
tb_bike = pd.read_csv('data/tb_bikesharing.csv')

In [ ]:
tb_bike.info()

In [ ]:
sns.kdeplot(data = tb_bike, x = 'cnt', hue = 'workingday', common_norm = False)

In [ ]:
sns.boxplot(data = tb_bike, y = 'cnt', x = 'workingday')

In [ ]:
tb_bike_fds = tb_bike[tb_bike['workingday'] == 0].copy()
tb_bike_sem = tb_bike[tb_bike['workingday'] == 1].copy()

### Assumption check 

- [ ]  The samples are independently and randomly drawn

**AND**

- [ ]  The distribution of the residuals between the two groups should follow the normal distribution

**AND**

- [ ]  The variances between the two groups are equal

**OR**

- [ ]  Approximately equal sample sizes

#### Normal?

In [ ]:
plt.figure(figsize=(10,10))
stats.probplot(tb_bike_fds['cnt'], plot=plt)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
stats.probplot(tb_bike_sem['cnt'], plot=plt)
plt.show()

#### Equal variance or equal sample size?

In [ ]:
tb_bike.groupby('workingday')['cnt'].var()

In [ ]:
stats.levene(tb_bike_fds['cnt'], tb_bike_sem['cnt'])

In [ ]:
tb_bike['workingday'].value_counts()

Os grupos estão desbalanceados e a variância é diferente! Será que podemos utilizar um teste-t? Nas palavras de Richard Nixon: <i>"We could do it... but it would be wrong"</i>. Como temos amostras grandes (n>>30) podemos rebalancear os dados!

<i>Ressalva: se as amostras tem uma proporção de 1,5:1 (300:200 pontos, por exemplo), podemos proceder sem grandes problemas.</i>

In [ ]:
tb_bike_fds_smp = tb_bike_fds.sample(231, random_state=42).reset_index(drop=True)

In [ ]:
tb_bike_fds_smp.shape

In [ ]:
stats.ttest_ind(tb_bike_fds_smp['cnt'], tb_bike_sem['cnt'])

# One tailed-side t-test

A hipótese nula padrão no teste-t é que a média das duas amostra é igual. Rejeitar esta hipótese significa afirmar que a média de uma amostra é maior ou menor que a média da outra amostra. No entanto, muitas vezes nos interessa determinar o desvio da média da amostra apenas em um sentido. Neste podemos utilizar o teste-t unilateral - one a hipótese nula **H0** é que **A <= B** OU  **A >= B**.

Alguns exemplos práticos: uma indústria de fertilizantes quer saber se seu novo produto aumenta a produtividade por hectar de milho plantado, um agência de comunicação deseja saber se um novo banner aumenta a taxa de cliques em um website, um cientista de dados deseja saber se seu novo sistema de recomendação aumenta as vendas.

In [ ]:
tb_bike['temp_classif'] = np.where(tb_bike['temp'] > np.mean(tb_bike['temp']), 1, 0)

In [ ]:
sns.boxplot(data = tb_bike, x = 'temp_classif', y = 'cnt')

In [ ]:
tb_bike_quente = tb_bike[tb_bike['temp_classif'] == 1].copy()
tb_bike_frio = tb_bike[tb_bike['temp_classif'] == 0].copy()

In [ ]:
tb_bike_frio.shape

In [ ]:
tb_bike_quente.shape

In [ ]:
stats.ttest_ind(tb_bike_quente['cnt'], tb_bike_frio['cnt'], alternative = 'greater')

In [ ]:
stats.ttest_ind(tb_bike_quente['cnt'], tb_bike_frio['cnt'])

# ANOVA

E se quisermos comparar a média de mais de um grupo? O método ANOVA testa a hipótese nula onde todas as amostras foram feitas da mesma população, ou seja, que a média dos diferentes grupos é identica. Rejeitar a hipótese nula significa dizer que **pelo menos um grupo não é uma amostra da mesma população, ou seja, pelo menos a média de um grupo é diferente dos grupos restantes**

In [ ]:
tb_bike['estacao'] = tb_bike['season'].map({
    1: 'Inverno',
    2: 'Primavera',
    3: 'Verão',
    4: 'Outono'
})

In [ ]:
sns.boxplot(data = tb_bike, x = 'estacao', y = 'cnt')

In [ ]:
tb_bike_season = [tb_bike[tb_bike['season'] == i]['cnt'] for i in range(1, 5)]

In [ ]:
stats.f_oneway(tb_bike_season[0], tb_bike_season[1], tb_bike_season[2],
               tb_bike_season[3])

E se quisermos saber a proporção entre os grupos?

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [ ]:
tukey_fit = pairwise_tukeyhsd(endog = tb_bike['cnt'], groups = tb_bike['estacao'], alpha = 0.05)

In [ ]:
tukey_fit.summary()